In [1]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import pandas as pd
import re

class PreprocessingText:
    def __init__(self):
        self.slang_list = [kamus.strip('\n').strip('\r') for kamus in open('./daftar_slangwords.txt')]
        self.stopword_list = [line.strip('\n')for line in open('./daftar_stopwords.txt')]
        
    # Case Folding
    def CaseFolding(self, tweets):
        tweets = tweets.lower()
        
        return tweets
    
    # Cleansing
    def Cleansing(self, tweets):
        # Menghilangkan mention, hashtag, character reference
        tweets = re.sub('[@#&][A-Za-z0-9_]+'," ", tweets)

        # Menghilangkan Tautan
        tweets = re.sub("\w+:\/\/\S+"," ", tweets)

        # Menghilangkan tanda baca
        tweets = re.sub('[()!?;,]', ' ', tweets)
        tweets = re.sub('\[.*?\]',' ', tweets)

        # Menghilangkan tanda selain huruf
        tweets = re.sub("[^a-z]"," ", tweets)

        # Menghilangkan spasi lebih dari 1
        tweets = ' '.join(tweets.split())
        
        return tweets
    
    # Replace Slangwords
    def Slangwords(self, tweets):
        for line in self.slang_list:
            slangword, unslang = line.strip().split(":")
            if slangword in tweets.split(" "):
                tweets = tweets.replace(slangword + " ", unslang + " ")
                tweets = tweets.replace(" " + slangword, " " + unslang)
                
        return tweets
    
    # Stopwords Removal
    def StopwordsRemoval(self, tweets):
        stopwords_list = list(map(lambda x:x.strip(),list(self.stopword_list)))
        tweets = tweets.split()
        tweets = [w for w in tweets if not w in stopwords_list]
        tweets = " ".join(word for word in tweets)
        
        return tweets
    
    # Tokenization
    def Tokenization(self, tweets):
        tweets = tweets.split()
        
        return tweets
    
    # Normalization (Stemming)
    def Stemming(self, tweets):
        factory = StemmerFactory()
        stemmer = factory.create_stemmer()
        do = []
        for w in tweets:
            dt = stemmer.stem(w)
            do.append(dt)
        d_clean = []
        d_clean = " ".join(do)
        print(d_clean)
        
        return d_clean

In [2]:
df = pd.read_csv("Final_Sebelum_Kanjuruhan.csv", sep=';', encoding='unicode_escape')
df

,id,conversation_id,created_at,user_id,username,label,tweet
0,"1,56E+18","1,56E+18",2022-08-31 06:58:15 SE Asia Standard Time,"1,43E+18",pinem3paska,Negatif,@Indostransfer Harapan saya Agar Di Sumatera ...
1,"1,56E+18","1,56E+18",2022-08-31 06:54:57 SE Asia Standard Time,968410669,bhaniie_bsf,Negatif,@ocehangaluh Haha gimana mau degradasi orang d...
2,"1,56E+18","1,56E+18",2022-08-31 06:53:15 SE Asia Standard Time,116308596,awaveiro,Negatif,"Detail banget woy, sampe pager betis lompat aj..."
3,"1,56E+18","1,56E+18",2022-08-31 06:48:16 SE Asia Standard Time,"1,45E+18",loyalisgaruda,Negatif,@abdulra55101674 @sahaaat_ @Vand0e @OrenDepok ...
4,"1,56E+18","1,56E+18",2022-08-31 06:44:19 SE Asia Standard Time,"1,27E+18",omah_balbalan,Positif,Kenapa sore? PSSI dan LIB harus tahu ini.
...,...,...,...,...,...,...,...
447,"1,48E+18","1,48E+18",2022-01-01 06:02:38 SE Asia Standard Time,"1,12E+18",fachsetiawan,Positif,@lisadepe Ga salah dong gw ngidolain Ajax keti...
448,"1,48E+18","1,48E+18",2022-01-01 06:02:38 SE Asia Standard Time,"7,08E+17",dulqowi,Positif,"@fedriza @PSSI Idem bro, sementara ini dulu aj..."
449,"1,48E+18","1,48E+18",2022-01-01 06:02:04 SE Asia Standard Time,"1,28E+18",ganesha010180,Negatif,"@SiaranBolaLive Ga habis pikir, PSSI menggunak..."
450,"1,48E+18","1,48E+18",2022-01-01 05:59:26 SE Asia Standard Time,"1,33E+18",indones30065690,Negatif,@tirta_cipeng @PSSI Perusak Sepakbola Sejati I...


In [5]:
pretext = PreprocessingText()

df_casefolding = df.copy()
df_casefolding['tweet'] = df_casefolding['tweet'].apply(pretext.CaseFolding)

df_cleansing = df_casefolding.copy()
df_cleansing['tweet'] = df_cleansing['tweet'].apply(pretext.Cleansing)

df_slangwords = df_cleansing.copy()
df_slangwords['tweet'] = df_slangwords['tweet'].apply(pretext.Slangwords)

df_stopwords = df_slangwords.copy()
df_stopwords['tweet'] = df_stopwords['tweet'].apply(pretext.StopwordsRemoval)

df_token = df_stopwords.copy()
df_token['tweet'] = df_token['tweet'].apply(pretext.Tokenization)

df_stemming = df_token.copy()
df_stemming['tweet'] = df_stemming['tweet'].apply(pretext.Stemming)

dataAll = {'Raw Data': df, 'Case Folding': df_casefolding, 'Cleansing': df_cleansing, 
       'Slangwords': df_slangwords, 'StopWords': df_stopwords, 'Tokenizing': df_token,
       'Stemming': df_stemming}

writer = pd.ExcelWriter('./Data_Sebelum_Preprocessed.xlsx', engine='xlsxwriter', mode='w')

for data_sheet in dataAll.keys():
    dataAll[data_sheet].to_excel(writer, sheet_name=data_sheet, index=False)

writer.close()

print("\n\n\nData Preprocessed Saved!!!")

harap sumatera utara kalimantan iri jaya tayang tanding jawa
mundur orang selamat pssi
detail pager betis lompat pikir
risiko hadap malam rawan celaka teman teman saudara korban tanding terlakamu malam marah marah jadwal malam
sore pssi lib
ketua pssi mochamad iriawan stadion piala dunia u tetap enam stadion kurang
jam tayang malam ubah evaluasi tutup mulut admin jatuh korban
temu sponsor pssi pasti piala indonesia gelar
hormat beku psim jogja konsekuensi dukung nyawa hilang sepak bola harga nyawa manusia adil damai
api asap bom takut psm makassar denda komdis pssi
periksa fifa pssi stadion piala dunia u kurang
jalas pikir benang pendek
jakarta panitia pikir lambat main malam
pssi stadion piala dunia u kurang
promosi kasta tinggi litidak kait siap baik ayo kelas profesional kait beli tiket terus
atur saing tim menang p ketua mesti strategi syarat juara aff kalah vietnam thailand anggap peringkat vietnam kerja keras
tanggap pssi aff nyata vietnam thailand langgar main adil piala aff u
j

KeyboardInterrupt: 

In [4]:
import pandas as pd

data = pd.read_csv("Final_Sesudah_Kanjuruhan.csv", sep=';', encoding='unicode_escape')
# data = pd.read_csv("Final_Sesudah_Kanjuruhan.csv", sep=';', encoding='unicode_escape')
data

,id,conversation_id,created_at,user_id,username,label,tweet
0,"1,58E+18","1,58E+18",2022-10-16 05:18:19 SE Asia Standard Time,"1,49E+18",alhusaini_asror,Negatif,"@mohmahfudmd Benar dan Betul.Ketua PSSI,mundur..."
1,"1,58E+18","1,58E+18",2022-10-16 05:13:51 SE Asia Standard Time,1442933096,metmalamminggu,Negatif,"@iIhamzada Dilarang intervensi gara"" aturan fi..."
2,"1,58E+18","1,58E+18",2022-10-16 05:06:34 SE Asia Standard Time,"7,88E+17",onedayasmine,Negatif,@gibran_tweet Mas.. jadi ketua PSSI yaaa Kalau...
3,"1,58E+18","1,58E+18",2022-10-16 05:03:44 SE Asia Standard Time,4812772283,harryunited05,Negatif,@medioclubID @PSSI mana orangnya itu2 aja lagi...
4,"1,58E+18","1,58E+18",2022-10-16 05:00:24 SE Asia Standard Time,"1,31E+18",ariflabmed,Positif,@medioclubID @PSSI Yunus Nusi akan dikenang se...
...,...,...,...,...,...,...,...
1233,"1,58E+18","1,58E+18",2022-10-07 11:47:59 SE Asia Standard Time,521254614,bodoamath,Negatif,lho kapolda sama ketua pssi nya mana?
1234,"1,58E+18","1,58E+18",2022-10-07 11:47:24 SE Asia Standard Time,"1,57E+18",crypto34209511,Negatif,@PSSI HAK DAN KEWAJIBAN PENONTON HARUS DI INGA...
1235,"1,58E+18","1,58E+18",2022-10-07 11:43:10 SE Asia Standard Time,2156012929,gbrand_9127,Negatif,@CNNIndonesia PSSI juga gk standar FIFA.
1236,"1,58E+18","1,58E+18",2022-10-07 11:41:33 SE Asia Standard Time,480858414,grrraargh,Negatif,@ainurohman Dia pikir PSSI cuma urus timnas


In [25]:
tweets_positive = [t for t in data["label"] if t == "Positif"]
tweets_negative = [t for t in data["label"] if t == "Negatif"]

print("Hasil Sentimen Analisis")
print("Positif : ", len(tweets_positive), "({}%)".format(100*len(tweets_positive)/len(data)))
print("Negatif : ", len(tweets_negative), "({}%)".format(100*len(tweets_negative)/len(data)))
print("TOTAL : ", len(data))

Hasil Sentimen Analisis
Positif :  619 (50.0%)
Negatif :  619 (50.0%)
TOTAL :  1238
